<a href="https://colab.research.google.com/github/lhwong/airbnb-analytics/blob/main/paidout.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
import pandas as pd
from google.colab import files
import io

uploaded = files.upload()


df2 = pd.read_csv(io.BytesIO(uploaded['airbnb_tax_01_2023-08_2023.csv']), parse_dates=['Date', 'Start Date'])



df2.loc[df2["Listing"] == "Level35 Ceylonz Suites overlooking KLCC, KL Tower", "Listing"] = 'R'
df2.loc[df2["Listing"] == "Level 35 Ceylonz Suites overlooking KLCC, KL Tower", "Listing"] = 'L'

result = df2.loc[df2['Type'] == "Reservation", 'Amount'].sum()

print("Total Paid Out Amount: ",result)

result = df2.loc[(df2['Type'] == "Reservation") & (df2['Listing'] == "L"), 'Amount'].sum()

print("Total Paid Out Amount for L: ",result)


result = df2.loc[(df2['Type'] == "Reservation") & (df2['Listing'] == "R"), 'Amount'].sum()

print("Total Paid Out Amount for R: ",result)



df2.groupby(df2.Date.dt.month)['Amount'].sum()
print(df2)


Saving airbnb_tax_01_2023-08_2023.csv to airbnb_tax_01_2023-08_2023.csv
Total Paid Out Amount:  28308.949999999997
Total Paid Out Amount for L:  12270.960000000001
Total Paid Out Amount for R:  16037.990000000003
          Date               Type Confirmation Code Start Date  Nights  \
0   2023-08-07        Reservation        HMKCHXYHAZ 2023-08-06       1   
1   2023-08-05        Reservation        HMZTYHCPRF 2023-08-04       2   
2   2023-08-04        Reservation        HM3ZWZSTH3 2023-08-03       4   
3   2023-08-02  Resolution Payout        HMC9THPFSC 2023-08-01       3   
4   2023-08-02        Reservation        HMC9THPFSC 2023-08-01       3   
..         ...                ...               ...        ...     ...   
101 2023-05-01        Reservation        HMXER3259Z 2023-04-30       1   
102 2023-05-01        Reservation        HMXNA2Q9S4 2023-04-30       1   
103 2023-04-30        Reservation        HMCQMMSQ8F 2023-04-29       1   
104 2023-04-29        Reservation        HMA38X